In [1]:
import sqlite3 as sql
import pandas

In [2]:
# connect to database
conn = sql.connect('NF_player_data.db')

In [3]:
# list table names
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
table_list = cursor.fetchall()

In [4]:
# Exclude firepower data tables
nf_tables = []
for i in table_list:
    for j in i:
        if j.find('NF_player') != -1:
            nf_tables.append(j)
display(nf_tables)

['NF_player_07072021',
 'NF_player_07142021',
 'NF_player_07212021',
 'NF_player_07282021']

In [5]:
# Extract newest player data table name
player_data = nf_tables[-1]

In [6]:
def lucky_bastard(season):
    firepower = 'firepower_season' + str(season)
    player_id = firepower + '.id'
    attack = ',' + firepower + '.highest_atk,'
    fleet = player_data + '.fleet,'
    squad = player_data + '.squad,'
    win_rate = player_data + '.win_rate '
    query_line = 'select ' + player_id + attack + fleet + squad + win_rate + 'from ' + firepower + ' LEFT JOIN ' + player_data + ' on ' + player_id + '=' + player_data + '.id order by highest_atk desc limit 25'
    print('\n' + '\033[1m' + '\033[92m' + 'Season ' + str(season) + ' Luckiest Players' + '\033[0m')
    print('\033[1m' + '\033[92m' + '火力王第' + str(season) + '期最幸运玩家' + '\033[0m')
    result = pandas.read_sql(query_line,conn)
    result['win_rate'] = pandas.Series(["{0:.1f}%".format(val * 100) for val in result['win_rate']], index = result.index)
    result.index += 1
    result = result.rename(columns = {'id':'账号', 'highest_atk':'最高攻击', 'fleet':'舰队', 'squad':'小队', 'win_rate':'胜率'})
    return result

In [7]:
def bastards(season):
    firepower = 'firepower_season' + str(season)
    player_id = firepower + '.id'
    attack = ',' + firepower + '.highest_atk,'
    total_atk = firepower + '.total_atk,'
    fleet = current_data + '.fleet,'
    squad = current_data + '.squad,'
    base_battles = base_data + '.battles as base_battles,'
    current_battles = current_data + '.battles as current_battles,'
    win_rate = current_data + '.win_rate '
    query_line = 'select ' + player_id + attack + total_atk + fleet + squad + base_battles + current_battles + win_rate + 'from ' + firepower + ' LEFT JOIN ' + current_data + ' on ' + player_id + '=' + current_data + '.id ' + 'left join ' + base_data + ' on ' + player_id + '=' + base_data + '.id'
    result = pandas.read_sql(query_line,conn)
    result = result.dropna()
    result['battles'] = result['current_battles']-result['base_battles']
    result = result[result.battles > 50]
    result['average_atk'] = round(result['total_atk']/(result['battles']*weights))
    result['average_atk'] = result['average_atk'].astype('int')
    keeps = ['id', 'highest_atk', 'average_atk', 'fleet', 'squad', 'win_rate']
    result = result[keeps]
    result = result.sort_values(by=['average_atk'], ascending=False)
    result['win_rate'] = pandas.Series(["{0:.1f}%".format(val * 100) for val in result['win_rate']], index = result.index)
    result.reset_index(drop=True, inplace=True)
    result.index += 1
    return result

def true_bastards(result, season):
    print('\n' + '\033[1m' + '\033[92m' + 'Season ' + str(season) + ' Hard Carrying Players' + '\033[0m')
    print('\033[1m' + '\033[92m' + '火力王第' + str(season) + '期场均伤害最高的玩家' + '\033[0m')
    result = result.rename(columns = {'id':'玩家账号', 'highest_atk':'最高攻击', 'average_atk':'场均攻击', 'fleet':'当前舰队', 'squad':'所属小队', 'win_rate':'当前胜率'})
    return result.head(25)

def some_bastards(result, season, who):
    print('\n' + '\033[1m' + '\033[92m' + 'Season ' + str(season) + ' Selective Players Summary' + '\033[0m')
    print('\033[1m' + '\033[92m' + '火力王第' + str(season) + '期个别玩家的场均输出' + '\033[0m')
    yesyou = result[result['id'].isin(who)]
    yesyou = yesyou.sort_values(by=['average_atk'], ascending=False)
    yesyou = yesyou.rename(columns = {'id':'玩家账号', 'highest_atk':'最高攻击', 'average_atk':'场均攻击', 'fleet':'当前舰队', 'squad':'所属小队', 'win_rate':'当前胜率'})
    return yesyou

In [11]:
# S12 Linear Estimation of Total Battles: dataset and weight scalar
base_data = 'NF_player_07072021'
current_data = 'NF_player_07282021'
weights= 32/21

# Season 12 Top 25 Heavy Carriers
monsters = bastards(12)
display(true_bastards(monsters, 12))

# Selected Players
checklist = ['laoji', 'z11285208', 'daiyinjj', 'tjdy', 'shu123', 'w2743882', 'yahhor', 'alwy', 'zjjd156', 'frankhjh', 'yxhysh', 
            'liming', 'hippo666', 'wololoooooo', 'odmr1', 'pantherelite', 'iloveasu', 'fanxiao008']
display(some_bastards(monsters, 12, checklist))


Season 12 Hard Carrying Players
火力王第12期场均伤害最高的玩家


,玩家账号,最高攻击,场均攻击,当前舰队,所属小队,当前胜率
1,berserker120,442319,219827,-,-,65.1%
2,kenny1337,428717,207511,中华联合舰队,-,67.5%
3,tskspree,495178,202815,兔子.CHN.,World,68.1%
4,huitainan_a,416109,188262,M.F Fleet,M.F共和国之鹰,66.7%
5,znfecu,506605,188215,-,勇敢天下,71.5%
6,dcldcldcl,337359,180274,-,-,60.0%
7,snaiper7,414460,178847,TeamKorea,Korea,63.3%
8,bin2014,321570,173615,北方舰队,THOR,62.0%
9,jdh998877,425125,172154,兔子.CHN.,-,55.8%
10,quendil,438748,169948,TEAM PRO BOTS,-,61.9%



Season 12 Selective Players Summary
火力王第12期个别玩家的场均输出


,玩家账号,最高攻击,场均攻击,当前舰队,所属小队,当前胜率
37,daiyinjj,404412,123172,中华联合舰队,中华联合一队,58.7%
52,shu123,268884,115666,深海舰队,扬帆起航,55.0%
57,z11285208,389516,114733,M.F Fleet,M.F共和国之鹰,64.5%
74,wololoooooo,315319,107520,-,-,62.9%
83,iloveasu,282967,105235,Dango Daikazoku,-,58.4%
143,odmr1,417525,90977,-,-,62.4%
148,pantherelite,295532,90354,-,-,56.0%
164,fanxiao008,338628,86882,帝国天雷舰队,-,58.3%
207,alwy,355373,79084,兔子.CHN.,-,55.2%
283,yxhysh,294771,69961,深海舰队,-,50.5%


In [9]:
# Current Season
lucky_bastard(12)


Season 12 Luckiest Players
火力王第12期最幸运玩家


,账号,最高攻击,舰队,小队,胜率
1,znfecu,506605,-,勇敢天下,71.5%
2,tskspree,495178,兔子.CHN.,World,68.1%
3,huha9999,488065,兔子.CHN.,World,52.5%
4,pashi001,479787,-,-,52.1%
5,snoopy1234,464630,兔子.CHN.,Hermit,57.9%
6,shoulingchao,463565,深海舰队,-,50.0%
7,xxshowtimexx,460504,-,-,67.5%
8,schoko,459518,TEAM PRO BOTS,$CHMUTZ,60.7%
9,meifutuzi,459137,兔子.CHN.,-,59.4%
10,ac16051,458868,BAMF FLEET,Legacy,54.3%


In [10]:
# Previous Seasons
for i in range(1,12):
    display(lucky_bastard(i))


Season 1 Luckiest Players
火力王第1期最幸运玩家


,账号,最高攻击,舰队,小队,胜率
1,yyss2221,499147,海风舰队,海風奔馳,74.9%
2,painlesskiss,489537,TEAM PRO BOTS,-,65.6%
3,xxooxooxx,482400,兔子.CHN.,-,61.1%
4,xxvetalxx,456120,Russian Alliance,Makarov,62.6%
5,jinzhiyuye,445800,中华联合舰队,勛章,52.1%
6,mpl252300,445697,中华联合舰队,-,48.7%
7,gumama1314_a,444787,兔子.CHN.,-,54.8%
8,z11285208,443200,M.F Fleet,M.F共和国之鹰,64.5%
9,meifutuzi,439789,兔子.CHN.,-,59.4%
10,liming,438943,中华联合舰队,-,64.9%



Season 2 Luckiest Players
火力王第2期最幸运玩家


,账号,最高攻击,舰队,小队,胜率
1,acepilot12,651356,兔子.CHN.,World,52.7%
2,xxooxooxx,631765,兔子.CHN.,-,61.1%
3,playab,579354,北方舰队,-,55.2%
4,ccpcwangbing,535611,北方舰队,THOR,55.0%
5,zatti,523831,Reiwa Fleet,-,61.4%
6,fuiyun,508220,齐云舰队,-,51.3%
7,quendil,496742,TEAM PRO BOTS,-,61.9%
8,e5234,483401,-,-,56.3%
9,dreamcaca0,480879,兔子.CHN.,World,64.9%
10,huitainan_a,473685,M.F Fleet,M.F共和国之鹰,66.7%



Season 3 Luckiest Players
火力王第3期最幸运玩家


...

,账号,最高攻击,舰队,小队,胜率
1,sanu4,526585,Russian Alliance,Makarov,54.5%
2,wswindy19811,526202,-,-,56.8%
3,zxziihanying,494729,-,-,61.4%
4,daiyinjj,477403,中华联合舰队,中华联合一队,58.7%
5,zjjd1001,473435,-,-,51.5%
6,zatti,469435,Reiwa Fleet,-,61.4%
7,w2f6q3i7,467246,兔子.CHN.,-,58.8%
8,ly19850623,466923,中华联合舰队,远征军,54.7%
9,jinzhiyuye,464214,中华联合舰队,勛章,52.1%
10,ak47ok,460559,中华联合舰队,中华联合二队,58.7%



Season 10 Luckiest Players
火力王第10期最幸运玩家


,账号,最高攻击,舰队,小队,胜率
1,zatti,555441,Reiwa Fleet,-,61.4%
2,navy0909_a,493733,007舰队,-,54.1%
3,xxshowtimexx,492248,-,-,67.5%
4,fanxiao008,491230,帝国天雷舰队,-,58.3%
5,ac16051,490260,BAMF FLEET,Legacy,54.3%
6,zxziihanying,481340,-,-,61.4%
7,znfecu,476318,-,勇敢天下,71.5%
8,gumama1314_a,474970,兔子.CHN.,-,54.8%
9,ygx1982as207,471450,-,北纬37度,59.3%
10,berserker120,468860,-,-,65.1%



Season 11 Luckiest Players
火力王第11期最幸运玩家


,账号,最高攻击,舰队,小队,胜率
1,zatti,589934,Reiwa Fleet,-,61.4%
2,zxziihanying,543863,-,-,61.4%
3,znfecu,531804,-,勇敢天下,71.5%
4,mariuchiha,507635,None,None,nan%
5,aspid47,500447,Russian Alliance,Nakhimov,58.7%
6,liuyishou1,490206,尖刀连,-,62.6%
7,ac16051,469268,BAMF FLEET,Legacy,54.3%
8,fs4025paz,462018,-,-,66.5%
9,nf0055,461073,北方舰队,-,59.4%
10,huitainan_a,458625,M.F Fleet,M.F共和国之鹰,66.7%
